In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import os
from math import *

print("The current directory is: ")
print(os.getcwd())
if not os.getcwd().endswith("Abaqus-Hardening-Seq-2-Seq-Project"):
    # Move up two directories
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)
print("The current directory is: ")
print(os.getcwd())

The current directory is: 
/scratch/project_2008630/Abaqus-Hardening-Seq-2-Seq-Project
The current directory is: 
/scratch/project_2008630/Abaqus-Hardening-Seq-2-Seq-Project


In [7]:
from configs.chosen_project import *
from src.stage1_global_configs import *

chosen_project_path = "configs/global_config_CP1000_RD_20C.json"

global_configs = main_global_configs(chosen_project_path)

all_paths = global_configs['all_paths']
objectives = global_configs['objectives']


= Stage 1: Loading configs and all paths =

Welcome to Abaqus Seq2Seq flow curve calibration project

The configurations you have chosen: 

+--------------------------+-------------------------------------------------------------+
|      Global Configs      |                         User choice                         |
+--------------------------+-------------------------------------------------------------+
|         PROJECT          |                        CP1000_RD_20C                        |
|        OBJECTIVES        |      CHD2, CHD4, NDBR2p5, NDBR6, NDBR15, NDBR40, SH115      |
|       PROJECT_PATH       | /scratch/project_2008630/Abaqus-Hardening-Seq-2-Seq-Project |
|    TRAINING_DATA_PATH    |                 training_data/CP1000_RD_20C                 |
|         LOG_PATH         |                    log/CP1000_RD_20C.txt                    |
|       MODELS_PATH        |                     models/CP1000_RD_20C                    |
|  RESULTS_INIT_DATA_PATH  |            

# Loading the training data

In [8]:
import torch

training_data_path = all_paths['training_data_path']
models_path = all_paths['models_path']

initial_train_source_sequence_diff = torch.load(f"{training_data_path}/initial_train_source_diff_all.pt")
#initial_train_source_sequence_diff = torch.load(f"{training_data_path}/initial_train_source_original_all.pt")
initial_train_target_sequence_diff = torch.load(f"{training_data_path}/initial_train_target_diff_last.pt")

initial_test_source_sequence_diff = torch.load(f"{training_data_path}/initial_test_source_diff_all.pt")
#initial_test_source_sequence_diff = torch.load(f"{training_data_path}/initial_test_source_original_all.pt")
initial_test_target_sequence_diff = torch.load(f"{training_data_path}/initial_test_target_diff_last.pt")

# Convert them to float32

initial_train_source_sequence_diff = initial_train_source_sequence_diff.float()
initial_train_target_sequence_diff = initial_train_target_sequence_diff.float()
initial_test_source_sequence_diff = initial_test_source_sequence_diff.float()
initial_test_target_sequence_diff = initial_test_target_sequence_diff.float()

print(f"Shape of the training source sequence: {initial_train_source_sequence_diff.shape}")
print(f"Shape of the training target sequence: {initial_train_target_sequence_diff.shape}")
print(f"Shape of the testing source sequence: {initial_test_source_sequence_diff.shape}")
print(f"Shape of the testing target sequence: {initial_test_target_sequence_diff.shape}")

# Check if any of them has NaN or infinite values

print(f"Number of NaN values in train_source_sequence: {np.isnan(initial_train_source_sequence_diff).sum()}")
print(f"Number of NaN values in train_target_sequence: {np.isnan(initial_train_target_sequence_diff).sum()}")
print(f"Number of NaN values in test_source_sequence: {np.isnan(initial_test_source_sequence_diff).sum()}")
print(f"Number of NaN values in test_target_sequence: {np.isnan(initial_test_target_sequence_diff).sum()}")

print(f"Number of infinite values in train_source_sequence: {np.isinf(initial_train_source_sequence_diff).sum()}")
print(f"Number of infinite values in train_target_sequence: {np.isinf(initial_train_target_sequence_diff).sum()}")
print(f"Number of infinite values in test_source_sequence: {np.isinf(initial_test_source_sequence_diff).sum()}")
print(f"Number of infinite values in test_target_sequence: {np.isinf(initial_test_target_sequence_diff).sum()}")

# Ensure that all target_sequence are positive
print(f"Number of negative values in train_target_sequence: {(initial_train_target_sequence_diff < 0).sum()}")
print(f"Number of negative values in test_target_sequence: {(initial_test_target_sequence_diff < 0).sum()}")

Shape of the training source sequence: torch.Size([192, 99, 7])
Shape of the training target sequence: torch.Size([192, 80, 1])
Shape of the testing source sequence: torch.Size([64, 99, 7])
Shape of the testing target sequence: torch.Size([64, 80, 1])
Number of NaN values in train_source_sequence: 0
Number of NaN values in train_target_sequence: 0
Number of NaN values in test_source_sequence: 0
Number of NaN values in test_target_sequence: 0
Number of infinite values in train_source_sequence: 0
Number of infinite values in train_target_sequence: 0
Number of infinite values in test_source_sequence: 0
Number of infinite values in test_target_sequence: 0
Number of negative values in train_target_sequence: 0
Number of negative values in test_target_sequence: 0


In [9]:
# Ensure that the scale of the source sequence is correct
print(initial_train_source_sequence_diff[0][0])

tensor([0.0443, 0.0462, 0.0313, 0.0378, 0.0508, 0.0566, 0.0325])


In [10]:
# Ensure that the scale of the target sequence is correct
print(initial_train_target_sequence_diff[0][0])

tensor([7.6436])


In [11]:
# Define the device

training_env = "CSC" # Choose between "local" and "CSC"
if training_env == "local":
    device = "cpu"
else:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
# Function to process the data
def process_sequence(tensor):
    # Calculate the minimum across the batch size dimension
    mean_values = tensor.mean(dim=0).squeeze(1)
    print(mean_values.shape)
    print(mean_values[0])
    print(mean_values[-1])
    # Calculate the ratio of the first timestep to the last timestep
    ratio = mean_values[0] / mean_values[-1]
    
    return ratio

# Process train and test target sequences
train_ratio = process_sequence(initial_train_target_sequence_diff)
test_ratio = process_sequence(initial_test_target_sequence_diff)

# Output the results (you might want to format or log these depending on your use case)
print("Training Data:")
#print("Minimum Values:", train_min)
print("Ratio First to Last Timestep:", train_ratio)

print("\nTest Data:")
#print("Minimum Values:", test_min)
print("Ratio First to Last Timestep:", test_ratio)

torch.Size([80])
tensor(6.0415)
tensor(0.7763)
torch.Size([80])
tensor(6.0572)
tensor(0.7847)
Training Data:
Ratio First to Last Timestep: tensor(7.7822)

Test Data:
Ratio First to Last Timestep: tensor(7.7188)


We shall proceed to train the model with RMSE of linearly decreasing weights, where max_ratio_differ is 1

It is much more costly to regress wrong the first flow curve incremental changes than the last ones, since if the first ones are wrong, these errors would accumulate and totally shift the flow curve. You can set max_ratio_differ to a higher value than 1 to prioritize the first points accuracy

# Start training the model

In [41]:
from optimization.LSTM_helper import *
from optimization.LSTM import *

def train_LSTM(previous_model, current_model, previous_best_test_loss, 
               dropout, num_epochs, start_lr, end_lr, start_tf, end_tf, 
               weight_decay, max_ratio_differ):
    
    train_dataset = TensorDataset(initial_train_source_sequence_diff, initial_train_target_sequence_diff)
    test_dataset = TensorDataset(initial_test_source_sequence_diff, initial_test_target_sequence_diff)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
    
    # Parameters
    _, source_len, feature_size = initial_train_source_sequence_diff.shape
    _, target_len, label_size = initial_train_target_sequence_diff.shape
    
    hidden_size = 256
    num_layers = 3
    
    bidirectional = True  # Set this flag to True or False as needed
    use_attention = True  # Set this flag to True or False to enable/disable attention

    # Initialize model, loss function, and optimizer
    model = LSTMModel(feature_size, label_size,
                      source_len, target_len,
                      hidden_size, num_layers,
                      dropout=dropout,
                      bidirectional=bidirectional, 
                      use_attention=use_attention).to(device)
    
    criterion = RMSELoss(linear_weight=True, max_ratio_differ = max_ratio_differ)  # Use the custom RMSE loss
    optimizer = optim.Adam(model.parameters(), lr=start_lr, weight_decay=weight_decay)  # Adding L2 regularization

    if previous_model is not None:
        # Loading the best model from the previous training
        model.load_state_dict(torch.load(f"{models_path}/LSTM/initial/best_model_{previous_model}.pth"))
        best_test_loss = previous_best_test_loss
    else:
        best_test_loss = float('inf')
        
    # Count the number of parameters
    total_params = sum(p.numel() for p in model.parameters())
    print(f'The model has {total_params} parameters.')
    
    # Track the best model
    best_model_path = f"{models_path}/LSTM/initial/best_model_{current_model}.pth"
    
    # Lists to track train and test losses
    train_losses = []
    test_losses = []
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
    
        # Update learning rate
        current_lr = linear_lr_scheduler(optimizer, epoch, start_lr, end_lr, num_epochs)
        
        # Get the current teacher forcing probability from the scheduler
        teacher_forcing_prob = linear_teacher_forcing_scheduler(epoch, start_tf, end_tf, num_epochs)
    
        # teacher_forcing_prob = log_teacher_forcing_scheduler(epoch, start_tf, end_tf, num_epochs)
        for batch_idx, (source_batch, target_batch) in enumerate(train_loader):
            source_batch, target_batch = source_batch.to(device), target_batch.to(device)
    
            optimizer.zero_grad()
    
            # Forward pass
            outputs = model(source_batch, target_batch, teacher_forcing_prob)
            loss = criterion(outputs, target_batch)
    
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
    
            train_loss += loss.item()
    
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
    
        # Evaluate on test set
        model.eval()
        test_loss = 0.0
        with torch.no_grad():
            for source_batch, target_batch in test_loader:
                source_batch, target_batch = source_batch.to(device), target_batch.to(device)
    
                # Forward pass
                outputs = model(source_batch)
                loss = criterion(outputs, target_batch)
    
                test_loss += loss.item()
    
        test_loss /= len(test_loader)
        test_losses.append(test_loss)
    
        # Save the best model
        if test_loss < best_test_loss:
            print(f"New best test loss found: {test_loss}")
            best_test_loss = test_loss
            best_model = model
            torch.save(model.state_dict(), best_model_path)
    
        # Print progress
        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.6f}, Test Loss: {test_loss:.6f}, LR: {current_lr:.9f}, TF: {teacher_forcing_prob:.9f}')
    
    # Save the train and test loss lists as .npy files
    np.save(f'{models_path}/LSTM/initial/train_losses_{current_model}.npy', np.array(train_losses))
    np.save(f'{models_path}/LSTM/initial/test_losses_{current_model}.npy', np.array(test_losses))
    
    torch.save(model.state_dict(), f"{models_path}/LSTM/initial/last_model_{current_model}.pth")
    
    print('Training complete')
    print(f'Best model saved with test loss: {best_test_loss:.4f}')

In [10]:
train_LSTM(previous_model=None, current_model="1_train_diff", previous_best_test_loss=None, 
               dropout=0.01, num_epochs=1000, start_lr=0.0005, end_lr=0.0001, start_tf=1.0, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
New best test loss found: 1.96438729763031
New best test loss found: 1.569395661354065
New best test loss found: 1.5277657508850098
New best test loss found: 1.4824365377426147
New best test loss found: 1.448875069618225
New best test loss found: 1.3935898542404175
New best test loss found: 1.2904384136199951
New best test loss found: 1.2588348388671875
New best test loss found: 1.178313970565796
New best test loss found: 1.1742969751358032
New best test loss found: 1.1713061332702637
New best test loss found: 1.1598947048187256
New best test loss found: 1.1527824401855469
New best test loss found: 1.1252061128616333
New best test loss found: 1.059348702430725
New best test loss found: 1.0451511144638062
New best test loss found: 0.9592779278755188
New best test loss found: 0.957600474357605
New best test loss found: 0.8903669118881226
New best test loss found: 0.8578134179115295
New best test loss found: 0.8510509133338928
New best test loss found: 0.

In [12]:
train_LSTM(previous_model="1_train_diff", current_model="2_train_diff", previous_best_test_loss=0.22307966649532318, 
               dropout=0.01, num_epochs=1000, start_lr=0.0001, end_lr=0.00009, start_tf=0.9, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
New best test loss found: 0.22300595045089722
New best test loss found: 0.22093456983566284
New best test loss found: 0.2175670564174652
New best test loss found: 0.21564088761806488
New best test loss found: 0.2154640108346939
New best test loss found: 0.21432575583457947
Epoch [100/1000], Train Loss: 0.188309, Test Loss: 0.214830, LR: 0.000099010, TF: 0.810900000
New best test loss found: 0.21429575979709625
New best test loss found: 0.2121727019548416
New best test loss found: 0.21105393767356873
New best test loss found: 0.21041744947433472
New best test loss found: 0.20930373668670654
Epoch [200/1000], Train Loss: 0.179368, Test Loss: 0.210118, LR: 0.000098010, TF: 0.720900000
New best test loss found: 0.2054360806941986
New best test loss found: 0.2042628824710846
New best test loss found: 0.2034272700548172
New best test loss found: 0.19784128665924072
Epoch [300/1000], Train Loss: 0.177490, Test Loss: 0.211344, LR: 0.000097010, TF: 0.630900000


In [13]:
train_LSTM(previous_model="2_train_diff", current_model="3_train_diff", previous_best_test_loss=0.18034061789512634, 
               dropout=0.01, num_epochs=1000, start_lr=0.00009, end_lr=0.00008, start_tf=0.8, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.170512, Test Loss: 0.201285, LR: 0.000089010, TF: 0.720800000
Epoch [200/1000], Train Loss: 0.159867, Test Loss: 0.188728, LR: 0.000088010, TF: 0.640800000
New best test loss found: 0.17982041835784912
New best test loss found: 0.17970949411392212
New best test loss found: 0.1786954551935196
New best test loss found: 0.17828530073165894
New best test loss found: 0.1777874380350113
Epoch [300/1000], Train Loss: 0.165427, Test Loss: 0.201794, LR: 0.000087010, TF: 0.560800000
New best test loss found: 0.17594434320926666
New best test loss found: 0.17417269945144653
Epoch [400/1000], Train Loss: 0.152533, Test Loss: 0.180536, LR: 0.000086010, TF: 0.480800000
New best test loss found: 0.17373988032341003
Epoch [500/1000], Train Loss: 0.151989, Test Loss: 0.181349, LR: 0.000085010, TF: 0.400800000
Epoch [600/1000], Train Loss: 0.146105, Test Loss: 0.177452, LR: 0.000084010, TF: 0.320800000
New best test loss found: 0.17352387

In [15]:
train_LSTM(previous_model="3_train_diff", current_model="4_train_diff", previous_best_test_loss=0.16958774626255035, 
               dropout=0.01, num_epochs=1000, start_lr=0.00008, end_lr=0.00007, start_tf=0.7, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
New best test loss found: 0.16935108602046967
Epoch [100/1000], Train Loss: 0.150884, Test Loss: 0.187298, LR: 0.000079010, TF: 0.630700000
New best test loss found: 0.16916455328464508
Epoch [200/1000], Train Loss: 0.147715, Test Loss: 0.170569, LR: 0.000078010, TF: 0.560700000
New best test loss found: 0.1691361665725708
Epoch [300/1000], Train Loss: 0.142758, Test Loss: 0.178749, LR: 0.000077010, TF: 0.490700000
New best test loss found: 0.16820958256721497
New best test loss found: 0.16751275956630707
Epoch [400/1000], Train Loss: 0.145456, Test Loss: 0.185463, LR: 0.000076010, TF: 0.420700000
New best test loss found: 0.16738858819007874
Epoch [500/1000], Train Loss: 0.139610, Test Loss: 0.173520, LR: 0.000075010, TF: 0.350700000
New best test loss found: 0.16705431044101715
Epoch [600/1000], Train Loss: 0.157059, Test Loss: 0.205480, LR: 0.000074010, TF: 0.280700000
New best test loss found: 0.16629688441753387
Epoch [700/1000], Train Loss: 0.158

In [16]:
train_LSTM(previous_model="4_train_diff", current_model="5_train_diff", previous_best_test_loss=0.16587655246257782, 
               dropout=0.01, num_epochs=1000, start_lr=0.00007, end_lr=0.00006, start_tf=0.6, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.135981, Test Loss: 0.167610, LR: 0.000069010, TF: 0.540600000
Epoch [200/1000], Train Loss: 0.134816, Test Loss: 0.166328, LR: 0.000068010, TF: 0.480600000
Epoch [300/1000], Train Loss: 0.151629, Test Loss: 0.193673, LR: 0.000067010, TF: 0.420600000
Epoch [400/1000], Train Loss: 0.134984, Test Loss: 0.178426, LR: 0.000066010, TF: 0.360600000
New best test loss found: 0.1657775491476059
New best test loss found: 0.16546069085597992
Epoch [500/1000], Train Loss: 0.139568, Test Loss: 0.171504, LR: 0.000065010, TF: 0.300600000
Epoch [600/1000], Train Loss: 0.133448, Test Loss: 0.171899, LR: 0.000064010, TF: 0.240600000
Epoch [700/1000], Train Loss: 0.137172, Test Loss: 0.167897, LR: 0.000063010, TF: 0.180600000
New best test loss found: 0.16492798924446106
Epoch [800/1000], Train Loss: 0.137620, Test Loss: 0.190070, LR: 0.000062010, TF: 0.120600000
New best test loss found: 0.1648319959640503
Epoch [900/1000], Train Loss: 0.

In [17]:
train_LSTM(previous_model="5_train_diff", current_model="6_train_diff", previous_best_test_loss=0.1648319959640503, 
               dropout=0.001, num_epochs=1000, start_lr=0.00006, end_lr=0.00005, start_tf=0.5, end_tf=0.0, 
               weight_decay=1e-4, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.130370, Test Loss: 0.173240, LR: 0.000059010, TF: 0.450500000
New best test loss found: 0.16470131278038025
New best test loss found: 0.16411830484867096
Epoch [200/1000], Train Loss: 0.141180, Test Loss: 0.170561, LR: 0.000058010, TF: 0.400500000
New best test loss found: 0.1619386523962021
Epoch [300/1000], Train Loss: 0.133926, Test Loss: 0.170656, LR: 0.000057010, TF: 0.350500000
Epoch [400/1000], Train Loss: 0.131046, Test Loss: 0.167997, LR: 0.000056010, TF: 0.300500000
Epoch [500/1000], Train Loss: 0.127670, Test Loss: 0.167885, LR: 0.000055010, TF: 0.250500000
Epoch [600/1000], Train Loss: 0.125340, Test Loss: 0.162600, LR: 0.000054010, TF: 0.200500000
New best test loss found: 0.1619034707546234
Epoch [700/1000], Train Loss: 0.128817, Test Loss: 0.163309, LR: 0.000053010, TF: 0.150500000
New best test loss found: 0.16093207895755768
New best test loss found: 0.16067059338092804
Epoch [800/1000], Train Loss: 0.13

In [18]:
train_LSTM(previous_model="6_train_diff", current_model="7_train_diff", previous_best_test_loss=0.16067059338092804, 
               dropout=0.001, num_epochs=1000, start_lr=0.00005, end_lr=0.00004, start_tf=0.4, end_tf=0.0, 
               weight_decay=1e-4, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.123999, Test Loss: 0.163712, LR: 0.000049010, TF: 0.360400000
Epoch [200/1000], Train Loss: 0.124305, Test Loss: 0.167512, LR: 0.000048010, TF: 0.320400000
Epoch [300/1000], Train Loss: 0.122005, Test Loss: 0.164947, LR: 0.000047010, TF: 0.280400000
Epoch [400/1000], Train Loss: 0.124524, Test Loss: 0.166618, LR: 0.000046010, TF: 0.240400000
Epoch [500/1000], Train Loss: 0.122832, Test Loss: 0.165969, LR: 0.000045010, TF: 0.200400000
Epoch [600/1000], Train Loss: 0.121015, Test Loss: 0.163685, LR: 0.000044010, TF: 0.160400000
Epoch [700/1000], Train Loss: 0.120582, Test Loss: 0.165714, LR: 0.000043010, TF: 0.120400000
Epoch [800/1000], Train Loss: 0.117667, Test Loss: 0.168573, LR: 0.000042010, TF: 0.080400000
Epoch [900/1000], Train Loss: 0.117154, Test Loss: 0.164501, LR: 0.000041010, TF: 0.040400000
New best test loss found: 0.16009749472141266
New best test loss found: 0.15983977913856506
New best test loss found: 0.

In [21]:
train_LSTM(previous_model="7_train_diff", current_model="8_train_diff", previous_best_test_loss=0.15922307968139648, 
               dropout=0.01, num_epochs=1000, start_lr=0.00004, end_lr=0.00003, start_tf=0.3, end_tf=0.0, 
               weight_decay=1e-4, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.126518, Test Loss: 0.161672, LR: 0.000039010, TF: 0.270300000
New best test loss found: 0.1591106802225113
Epoch [200/1000], Train Loss: 0.128226, Test Loss: 0.168123, LR: 0.000038010, TF: 0.240300000
Epoch [300/1000], Train Loss: 0.118072, Test Loss: 0.162086, LR: 0.000037010, TF: 0.210300000
Epoch [400/1000], Train Loss: 0.120057, Test Loss: 0.162620, LR: 0.000036010, TF: 0.180300000
Epoch [500/1000], Train Loss: 0.117548, Test Loss: 0.163697, LR: 0.000035010, TF: 0.150300000
Epoch [600/1000], Train Loss: 0.113670, Test Loss: 0.162812, LR: 0.000034010, TF: 0.120300000
Epoch [700/1000], Train Loss: 0.119189, Test Loss: 0.165305, LR: 0.000033010, TF: 0.090300000
Epoch [800/1000], Train Loss: 0.118446, Test Loss: 0.169260, LR: 0.000032010, TF: 0.060300000
Epoch [900/1000], Train Loss: 0.110722, Test Loss: 0.163946, LR: 0.000031010, TF: 0.030300000
Epoch [1000/1000], Train Loss: 0.110336, Test Loss: 0.166198, LR: 0.0000300

In [14]:
train_LSTM(previous_model="8_train_diff", current_model="9_train_diff", previous_best_test_loss=0.1591106802225113, 
               dropout=0.001, num_epochs=1000, start_lr=0.00003, end_lr=0.00002, start_tf=0.2, end_tf=0.0, 
               weight_decay=1e-4, max_ratio_differ=1)

The model has 8955393 parameters.
New best test loss found: 0.15910851955413818
New best test loss found: 0.15864631533622742
Epoch [100/1000], Train Loss: 0.123078, Test Loss: 0.164400, LR: 0.000029010, TF: 0.180200000
Epoch [200/1000], Train Loss: 0.115607, Test Loss: 0.162899, LR: 0.000028010, TF: 0.160200000
Epoch [300/1000], Train Loss: 0.111396, Test Loss: 0.165795, LR: 0.000027010, TF: 0.140200000
Epoch [400/1000], Train Loss: 0.113481, Test Loss: 0.163575, LR: 0.000026010, TF: 0.120200000
Epoch [500/1000], Train Loss: 0.111031, Test Loss: 0.169433, LR: 0.000025010, TF: 0.100200000
Epoch [600/1000], Train Loss: 0.112176, Test Loss: 0.168567, LR: 0.000024010, TF: 0.080200000
Epoch [700/1000], Train Loss: 0.113898, Test Loss: 0.177234, LR: 0.000023010, TF: 0.060200000
Epoch [800/1000], Train Loss: 0.108568, Test Loss: 0.169621, LR: 0.000022010, TF: 0.040200000
Epoch [900/1000], Train Loss: 0.111286, Test Loss: 0.171190, LR: 0.000021010, TF: 0.020200000
Epoch [1000/1000], Train Los

In [16]:
train_LSTM(previous_model="9_train_diff", current_model="10_train_diff", previous_best_test_loss=0.15864631533622742, 
               dropout=0.01, num_epochs=1000, start_lr=0.0001, end_lr=0.00009, start_tf=1.0, end_tf=0.0, 
               weight_decay=1e-3, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.138576, Test Loss: 0.168090, LR: 0.000099010, TF: 0.901000000
New best test loss found: 0.15836161375045776
Epoch [200/1000], Train Loss: 0.142388, Test Loss: 0.181058, LR: 0.000098010, TF: 0.801000000
Epoch [300/1000], Train Loss: 0.147898, Test Loss: 0.163096, LR: 0.000097010, TF: 0.701000000
Epoch [400/1000], Train Loss: 0.144960, Test Loss: 0.180207, LR: 0.000096010, TF: 0.601000000
Epoch [500/1000], Train Loss: 0.133616, Test Loss: 0.167906, LR: 0.000095010, TF: 0.501000000
Epoch [600/1000], Train Loss: 0.140805, Test Loss: 0.175297, LR: 0.000094010, TF: 0.401000000
Epoch [700/1000], Train Loss: 0.165074, Test Loss: 0.217210, LR: 0.000093010, TF: 0.301000000
Epoch [800/1000], Train Loss: 0.137053, Test Loss: 0.181073, LR: 0.000092010, TF: 0.201000000
Epoch [900/1000], Train Loss: 0.137777, Test Loss: 0.179336, LR: 0.000091010, TF: 0.101000000
Epoch [1000/1000], Train Loss: 0.128488, Test Loss: 0.171980, LR: 0.000090

In [46]:
train_LSTM(previous_model="9_train_diff", current_model="10_train_diff", previous_best_test_loss=0.15836161375045776, 
               dropout=0.01, num_epochs=1000, start_lr=0.0001, end_lr=0.00009, start_tf=0.1, end_tf=0.0, 
               weight_decay=1e-5, max_ratio_differ=1)

The model has 8955393 parameters.
New best test loss found: 0.157598614692688
Epoch [100/1000], Train Loss: 0.152591, Test Loss: 0.165059, LR: 0.000099010, TF: 0.090100000
New best test loss found: 0.1567518711090088
Epoch [200/1000], Train Loss: 0.153309, Test Loss: 0.167484, LR: 0.000098010, TF: 0.080100000
New best test loss found: 0.15596969425678253
New best test loss found: 0.15532825887203217
New best test loss found: 0.15518268942832947
Epoch [300/1000], Train Loss: 0.128172, Test Loss: 0.163406, LR: 0.000097010, TF: 0.070100000
Epoch [400/1000], Train Loss: 0.117666, Test Loss: 0.164971, LR: 0.000096010, TF: 0.060100000
Epoch [500/1000], Train Loss: 0.140626, Test Loss: 0.185287, LR: 0.000095010, TF: 0.050100000
New best test loss found: 0.15482302010059357
Epoch [600/1000], Train Loss: 0.128237, Test Loss: 0.168049, LR: 0.000094010, TF: 0.040100000
Epoch [700/1000], Train Loss: 0.115749, Test Loss: 0.161059, LR: 0.000093010, TF: 0.030100000
Epoch [800/1000], Train Loss: 0.117

In [ ]:
train_LSTM(previous_model="10_train_diff", current_model="11_train_diff", previous_best_test_loss=0.15482302010059357, 
               dropout=0.01, num_epochs=1000, start_lr=0.00009, end_lr=0.00008, start_tf=0.1, end_tf=0.0, 
               weight_decay=1e-5, max_ratio_differ=1)

The model has 8955393 parameters.
Epoch [100/1000], Train Loss: 0.135617, Test Loss: 0.185379, LR: 0.000089010, TF: 0.090100000
Epoch [200/1000], Train Loss: 0.118424, Test Loss: 0.164631, LR: 0.000088010, TF: 0.080100000
Epoch [300/1000], Train Loss: 0.137331, Test Loss: 0.184222, LR: 0.000087010, TF: 0.070100000
Epoch [400/1000], Train Loss: 0.124519, Test Loss: 0.179539, LR: 0.000086010, TF: 0.060100000
